In [58]:
import numpy as np
from skimage import io, morphology
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import imageio

In [96]:
import numpy as np
from sklearn.linear_model import LinearRegression

def get_window(data, index, window_size=(32, 32, 32)):
    assert len(index) == 3, "Indices must be in 3-dimensional triplets (x,y,z)"

    x, y, z = index
    x_min = max(x - window_size[0] // 2, 0)
    x_max = min(x + window_size[0] // 2, data.shape[0])
    y_min = max(y - window_size[1] // 2, 0)
    y_max = min(y + window_size[1] // 2, data.shape[1])
    z_min = max(z - window_size[2] // 2, 0)
    z_max = min(z + window_size[2] // 2, data.shape[2])
    
    window = data[x_min:x_max, y_min:y_max, z_min:z_max]
    return window


def fit_3d_line(data):
    # Get the coordinates of the points
    coords = np.array(np.nonzero(data)).T
    values = data[data != 0]
    
    # Check if we have enough points to fit a line
    if len(coords) < 2:
        return None, None
    
    # Fit a 3D line
    reg = LinearRegression().fit(coords, values)
    slope = reg.coef_
    r_squared = reg.score(coords, values)
    
    return slope, r_squared

def get_slope(data, indices):
    result_dict = {}
    
    for index in indices:
        window = get_window(data, index)
        slope, r_squared = fit_3d_line(window)
        if slope is not None and r_squared is not None:
            result_dict[tuple(index)] = {'slope': slope, 'r_squared': r_squared}
    
    return result_dict

### Read Stack

In [78]:
import tifffile as tiff

# Read the TIFF stack
data = tiff.imread('output/thresh_stack.tif')

# Check the shape of the data
print(data.shape)


(188, 2044, 2042)


In [ ]:
# Resize

### Get Indices

In [90]:
# Find indices of positive mask
indices = np.where(data == 1) 
# Convert to list of triplets
indices_list = list(zip(indices[0], indices[1], indices[2]))

In [92]:
# Sanity check
print(len(indices_list))
print(indices_list[0])

1256635
(0, 1323, 770)


### Find Slopes

In [97]:
# Example usage:

result = get_slope(data, indices_list)
print(len(result))

1256635
